FIRST STEP: getting data from balance sheets

In [1]:
import pandas as pd
import numpy as np
import os
import json
# from sklearn.externals import joblib

oscwd = os.getcwd()

# Load dataframe with BALANCE SHEET data (assets, liabilities and equity)
df_evoldatos = pd.read_csv(os.path.join(oscwd, "data/dbo_Men_EvolDatos.txt"),
                            dtype = {
                                    'Entidad': str, 'Periodo': str, 
                                    'Cuenta': str, 'Saldo': float
                                    }
                                    # parse_dates=['Periodo'],
                            )

# Load dataframe with INCOME STATEMENT data
df_evolresul = pd.read_csv(os.path.join(oscwd, "data/dbo_Men_EvolResul.txt"),
                            dtype = {
                                    'Entidad': str, 'Periodo': str, 
                                    'Cuenta': str, 'Saldo': float
                                    }
                                    # parse_dates=['Periodo'],
                            )

# Load dataframe with SPECIAL data needed for personal loans
df_evolintdocpers = pd.read_csv(os.path.join(oscwd, "data/dbo_Men_IntDocPers.txt"),
                            dtype = {
                                    'Codent': str, 'FEC_INF': str, 
                                    'PARTIDA': str, 'SumaDeIMPORTE': float
                                    }
                                    # parse_dates=['Periodo'],
                            )

# Load dataframe with average weighted value of loan segments
df_avg_loan_seg_weighted_value = pd.read_csv(os.path.join(oscwd, "data/df_loan_seg_avg_wv.csv"),
                            dtype = {
                                    'bank': str, 'per': str, 
                                    'acc': str, 'weighted_value': float
                                    }
                                    # parse_dates=['Periodo'],
                            )

# Load dataframe with BALANCE SHEET data (assets, liabilities and equity)
df_evoldeu = pd.read_csv(os.path.join(oscwd, "data/dbo_Men_EvolDeu.txt"),
                            dtype = {
                                    'EENTIDAD': str, 'C_FEC_INF': str, 'D_SECTOR': str,
                                    'D_CARTERA': str, 'SumaDeN_SITTOT': float
                                    }
                                    # parse_dates=['Periodo'],
                            )


# Change to calculate different variations
df_evoldeu["D_SEC_CAR"] = [i.lower()[0:4] for i in df_evoldeu["D_CARTERA"]] # [df_evoldeu["D_CARTERA"][i].lower()[0:4]+df_evoldeu["D_SECTOR"][i].lower()[0:4] for i in range(len(df_evoldeu))]
df_evoldeu = df_evoldeu.iloc[:,[0,1,5,4]]
# df_evoldeu["D_SEC_CAR"] = ["00000000001" if i=="come" else "00000000002" for i in df_evoldeu["D_SEC_CAR"]]

'''car_sec_list = list(df_evoldeu["D_SEC_CAR"].unique())
dict_d_sec_car = dict((car_sec, car_sec_list.index(car_sec)) for car_sec in car_sec_list)'''

# Renaming columns
new_columns = ["bank", "date", "acc", "total"]
df_evoldatos.columns = new_columns
df_evolresul.columns = new_columns
df_evolintdocpers.columns = new_columns
df_avg_loan_seg_weighted_value.columns = new_columns
df_evoldeu.columns = new_columns

# Yearly column for later aggregation
df_evoldatos["per"] = [d[0:4] for d in df_evoldatos["date"]]
df_evolresul["per"] = [d[0:4] for d in df_evolresul["date"]]
df_evolintdocpers["per"] = [d[0:4] for d in df_evolintdocpers["date"]]
df_avg_loan_seg_weighted_value["per"] = [d[0:4] for d in df_avg_loan_seg_weighted_value["date"]]
df_evoldeu["per"] = [d[0:4] for d in df_evoldeu["date"]]

In [2]:
df_asset_total = df_evoldatos.loc[df_evoldatos.acc=="10000000000"]
df_asset_total = df_asset_total.groupby(["date", "acc","per"]).sum("total").reset_index()
df_asset_total = df_asset_total.groupby(["acc", "per"]).mean("total").reset_index()
df_asset_total["bank"] = "11111"
df_asset_total["acc"] = "10000000000_"+str(df_asset_total["bank"][0])
df_asset_total = df_asset_total.iloc[:,[3,0,1,2]]

We will be using operative banks (excluding those who have shut)

In [3]:
# Load dataframe with average weighted value of loan segments
df_entidad = pd.read_csv(os.path.join(oscwd, "data/ENTIDAD.txt"), encoding="utf-8",
                            dtype = {
                                    'EENTIDAD': str, 'CODNUM': int, 
                                    'NOMBRE CORTO': str, "VIGENTE": int, 
                                    "FECHA_ALTA": str, "BAJA": str
                                    }
                                    # parse_dates=['Periodo'],
                            )
df_entidad = df_entidad[["EENTIDAD", "CODNUM", "NOMBRE CORTO", "VIGENTE", "FECHA_ALTA", "BAJA"]]

In [4]:
# Load dataframe with operative banks
active = pd.DataFrame(df_entidad["EENTIDAD"][df_entidad["VIGENTE"]==1])
active.columns = ["bank"]

In [5]:
# Create list of operative banks that are actually in the balance sheet statement
banks = [a for a in active["bank"] if a in df_evoldatos["bank"].unique()]
# banks

In [6]:
gh_final = "report/test_step_by_step_clustering/Silhouette Method-MiniBatchKMeans-[2]-Robust Scaler-Robust Covariance.csv"
df_gh = pd.read_csv(gh_final, header=0)
df_gh.columns = ["bank_int", "group"]
display(type(df_gh["group"].max().item()))


dict_groups = {
    1: [11, 14, 29, 300],
    2: [143, 158, 384, 45030],
    3: [340, 44059, 44088, 44092, 44093, 44094, 44095, 44096, 44098, 44099]
}

for i in range(1,4):
    g = df_gh["group"].max().item() + 1
    banks_to_append = pd.DataFrame({
        "bank_int": dict_groups[i],
        "group": [g for b in range(len(dict_groups[i]))]
        })
    #print(banks_to_append["group"])
    df_gh = pd.concat([df_gh, banks_to_append])

#display(df_gh)


int

SECOND STEP: loading balance sheet accounts for each kpi from a json file

In [7]:
with open("data/kpi.json", "r") as kpi_json:
    kpi_ind = json.load(kpi_json)

num_list = list()
num_sign = list()
den_list = list()
den_sign = list()

for k in kpi_ind:
    num_list.append(kpi_ind[k]["num"]["n_acc"])
    num_sign.append(kpi_ind[k]["num"]["sign"])
    den_list.append(kpi_ind[k]["den"]["n_acc"])
    den_sign.append(kpi_ind[k]["den"]["sign"])

THIRD STEP: filtering, aggregating and concatenating data.

We will remove non operative banks and also aggregate the data: grouping and averaging BALANCE SHEET accounts, while adding accounts for INCOME STATEMENT

In [8]:
# Filtering by operative banks
df_evoldatos = df_evoldatos[df_evoldatos["bank"].isin(banks)]
df_evolresul = df_evolresul[df_evolresul["bank"].isin(banks)]
df_evolintdocpers = df_evolintdocpers[df_evolintdocpers["bank"].isin(banks)]
df_avg_loan_seg_weighted_value = df_avg_loan_seg_weighted_value[df_avg_loan_seg_weighted_value["bank"].isin(banks)]
df_evoldeu = df_evoldeu[df_evoldeu["bank"].isin(banks)]

df_evoldatos["bank_int"] = df_evoldatos["bank"].astype(int)
df_evolresul["bank_int"] = df_evolresul["bank"].astype(int)
df_evolintdocpers["bank_int"] = df_evolintdocpers["bank"].astype(int)
df_avg_loan_seg_weighted_value["bank_int"] = df_avg_loan_seg_weighted_value["bank"].astype(int)
df_evoldeu["bank_int"] = df_evoldeu["bank"].astype(int)

#display(df_evoldatos)

In [9]:
# Merge with group dataframe
df_evoldatos = pd.merge(df_evoldatos, df_gh, how="left", on="bank_int").drop(columns="bank_int")
df_evolresul = pd.merge(df_evolresul, df_gh, how="left", on="bank_int").drop(columns="bank_int")
df_evolintdocpers = pd.merge(df_evolintdocpers, df_gh, how="left", on="bank_int").drop(columns="bank_int")
df_avg_loan_seg_weighted_value = pd.merge(df_avg_loan_seg_weighted_value, df_gh, how="left", on="bank_int").drop(columns="bank_int")
df_evoldeu = pd.merge(df_evoldeu, df_gh, how="left", on="bank_int").drop(columns="bank_int")
#df_exp = df_evolresul.loc[(df_evolresul["acc"]=="00000560000")]
#df_asset = df_evoldatos.loc[(df_evoldatos["acc"]=="10000000000")]
#df_exp.to_csv("data/expenses.csv", sep=",", index=False)
#df_asset.to_csv("data/assests.csv", sep=",", index=False)
# display(df_evoldatos.loc[(df_evoldatos["group"]==0)&(df_evoldatos["acc"]=="10000000000")])

In [10]:
df_evoldatos_agg = df_evoldatos.groupby(['date', 'acc', 'per', 'group']).sum('total').reset_index()
df_evoldeu_agg = df_evoldeu.groupby(['date', 'acc', 'per', 'group']).sum('total').reset_index()
#display(df_evoldatos_agg.loc[(df_evoldatos_agg["group"]==0)&(df_evoldatos_agg["acc"]=="10000000000")&(df_evoldatos_agg["per"]=="2001")])

In [11]:
# Aggregating data
df_evoldatos_agg = df_evoldatos_agg.groupby(['group', 'acc', 'per']).mean('total')
df_evolresul_agg = df_evolresul.groupby(['group', 'acc', 'per']).sum('total')
df_evolintdocpers_agg = df_evolintdocpers.groupby(['group', 'acc', 'per']).mean('total')
df_evoldeu_agg = df_evoldeu_agg.groupby(['group', 'acc', 'per']).mean('total')

In [12]:
# Reseting indexes
df_evoldatos_agg = df_evoldatos_agg.reset_index()
df_evolresul_agg = df_evolresul_agg.reset_index()
df_evolintdocpers_agg = df_evolintdocpers_agg.reset_index()
df_evoldeu_agg = df_evoldeu_agg.reset_index()

In [13]:
# df_test = pd.concat([df_evoldatos_agg[df_evoldatos_agg["Cuenta"]=="10000000000"], df_asset_total], axis=1, keys="Anual")
df_asset = df_evoldatos_agg[df_evoldatos_agg["acc"]=="10000000000"]
df_asset["total"] = [df_asset_total["total"].loc[df_asset_total.index[df_asset_total["per"]==p][0]] for p in df_asset["per"]]
df_asset["acc"] = "10000000000_11111"

C:\Users\MEC\AppData\Local\Temp\ipykernel_24204\1280478950.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asset["total"] = [df_asset_total["total"].loc[df_asset_total.index[df_asset_total["per"]==p][0]] for p in df_asset["per"]]
C:\Users\MEC\AppData\Local\Temp\ipykernel_24204\1280478950.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asset["acc"] = "10000000000_11111"


In [14]:
# Concatenating and filtering out any data prior to ....
df_evol = pd.concat([df_evoldatos_agg, df_evolresul_agg, df_evolintdocpers_agg, df_asset, df_avg_loan_seg_weighted_value, df_evoldeu_agg])
# df_evol["bank_int"] = df_evol["bank"].astype(int)
# df_evol = pd.merge(df_evol, df_gh, how="left", on="bank_int")
start = "2001"
end = "2022"
df_evol = df_evol[((df_evol["per"])>=start)&(df_evol["per"]<=end)]
df_evol = df_evol.drop(columns="date")

display(df_evol.loc[(df_evol["group"]==0)&(df_evol["acc"]=="10000000000")&(df_evol["per"]=="2001")].head(10))

,group,acc,per,total,bank
0,0.0,10000000000,2001,483108.583333,NaN


FOURTH STEP: calculating kpis

In [15]:
per_list = [p for p in df_evol["per"].unique()]
acc_lists = [num_list[i]+den_list[i] for i in range(len(num_list))]
kpi_list = [kpi for kpi in kpi_ind.keys()]

# display(per_list, acc_lists, kpi_list)

df_ind = pd.DataFrame([], columns=["group", "per", "ind", "val"])

for i, acc_list in enumerate(acc_lists):
    for p in per_list:
        try:
            df_pivot = df_evol[(df_evol["per"]==p)&(df_evol["acc"].isin(acc_list))].pivot(index=["group"], columns="acc", values="total").reset_index()
            # df_pivot = df_pivot.fillna(0)
            df_pivot["num"] = (df_pivot[df_pivot.columns & num_list[i]] * [num_sign[i][num_list[i].index(acc_list)] for acc_list in df_pivot[df_pivot.columns & num_list[i]].columns]).sum(axis=1)
            df_pivot["den"] = (df_pivot[df_pivot.columns & den_list[i]] * [den_sign[i][den_list[i].index(acc_list)] for acc_list in df_pivot[df_pivot.columns & den_list[i]].columns]).sum(axis=1)
            df_pivot[kpi_list[i]] = df_pivot["num"] / df_pivot["den"]
            df_pivot[kpi_list[i]] = df_pivot[kpi_list[i]].replace([np.inf, -np.inf, ""], 0)
            df_pivot[kpi_list[i]] = df_pivot[kpi_list[i]].fillna(0)

            df_to_ind = pd.DataFrame({"group": df_pivot[df_pivot.columns[0]], "per": p, "ind": kpi_list[i], "val": df_pivot[kpi_list[i]]})

            df_ind = df_ind.append(df_to_ind)
        
        except Exception as e:
            print(e)
            pass

df_ind.to_csv("report/test_step_by_step_clustering/first_step_2_groups.csv", sep=",", index=False)

C:\Users\MEC\AppData\Local\Temp\ipykernel_24204\1703976133.py:14: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  df_pivot["num"] = (df_pivot[df_pivot.columns & num_list[i]] * [num_sign[i][num_list[i].index(acc_list)] for acc_list in df_pivot[df_pivot.columns & num_list[i]].columns]).sum(axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_24204\1703976133.py:15: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  df_pivot["den"] = (df_pivot[df_pivot.columns & den_list[i]] * [den_sign[i][den_list[i].index(acc_list)] for acc_list in df_pivot[df_pivot.columns & den_list[i]].columns]).sum(axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_24204\1703976133.py:22: FutureWarning: The frame.append method is deprecated and will 